In [73]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp

In [74]:
# Define initial conditions
L = 20
n = 64
N = n * n
beta = 1
D1, D2 = 0.1, 0.1
tspan = np.arange(0, 4.5, 0.5)

x = np.linspace(-L/2, L/2, n + 1)
x = x[:n]
y = np.linspace(-L/2, L/2, n + 1)
y = y[:n]
[X,Y] = np.meshgrid(x,y)
dx = L/n
dy = L/n
kx = (2 * np.pi / L) * np.hstack((np.linspace(0, n//2 - 1, n//2), np.linspace(-n//2, -1, n//2)))
ky = kx
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2

m = 1
U = np.zeros((len(x), len(y), len(tspan))) 
V = np.zeros((len(x), len(y), len(tspan)))
U[:, :, 0] = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j*Y) - (np.sqrt(X**2 + Y**2)))
V[:, :, 0] = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j*Y) - (np.sqrt(X**2 + Y**2)))

In [75]:
def lambda_A(U, V):
    A_2 = U**2 + V**2
    return 1 - A_2

def omega_A(U, V):
    A_2 = U**2 + V**2
    return -beta * A_2

def reaction_diffusion(t, uv, K2, n, N):
    u = np.reshape(uv[:N], (n, n))
    v = np.reshape(uv[N:], (n, n))
    u_ifft = ifft2(u)
    v_ifft = ifft2(v)
    U_reaction = lambda_A(u_ifft, v_ifft) * u_ifft - omega_A(u_ifft, v_ifft) * v_ifft
    V_reaction = omega_A(u_ifft, v_ifft) * u_ifft + lambda_A(u_ifft, v_ifft) * v_ifft
    U_reaction = fft2(U_reaction)
    V_reaction = fft2(V_reaction)
    
    U_diffusion = (-D1 * K2 * (u))
    V_diffusion = (-D2 * K2 * (v))

    dudt = (U_diffusion + U_reaction).flatten()
    dvdt = (V_diffusion + V_reaction).flatten()
    
    return np.concatenate([dudt, dvdt])

UV0 = np.concatenate([fft2(U[:,:,0]).reshape(N),fft2(V[:,:,0]).reshape(N)])


UVsol = solve_ivp(reaction_diffusion, (tspan[0], tspan[-1]), y0=UV0,
                  t_eval=tspan, args=(K2, n, N))
A1 = UVsol.y
print(A1)

[[ 24.94003847+0.00000000e+00j  12.73268299-1.93539634e-15j
   -1.38095598-5.57215725e-15j ... -64.02389647-9.70717753e-15j
  -67.76356741-7.75782520e-15j -61.18058974-1.09033738e-14j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389

In [76]:
# Define initial conditions
L = 10  
n = 30  

def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = np.arange(0,N+1)
		x = np.cos(np.pi*n/N).reshape(N+1,1) 
		c = (np.hstack(( [2.], np.ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = np.tile(x,(1,N+1))
		dX = X - X.T
		D = np.dot(c,1./c.T)/(dX+np.eye(N+1))
		D -= np.diag(np.sum(D.T,axis=0))
	return D, x.reshape(N+1)

D, x_cheb = cheb(n)
D[n, :] = 0
D[0, :] = 0
y_cheb = x_cheb.copy()
X_cheb, Y_cheb = np.meshgrid(x_cheb, y_cheb)

X = L * X_cheb
Y = L * Y_cheb

U = np.zeros((n+1, n+1))
V = np.zeros((n+1, n+1))
U = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
V = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))


Dxx = np.dot(D / L, D / L)
Dyy = Dxx
I = np.eye(len(Dxx))
Lap = np.kron(I, Dxx) + np.kron(Dyy, I)

def reaction_diffusion(t, uv):
    N = (n+1)**2
    u = uv[:N].reshape((n+1, n+1))
    v = uv[N:].reshape((n+1, n+1))

    U_reaction = lambda_A(u, v) * u - omega_A(u, v) * v
    V_reaction = omega_A(u, v) * u + lambda_A(u, v) * v
    
    U_diffusion = D1 * Lap.dot(u.flatten()).reshape((n+1, n+1))
    V_diffusion = D2 * Lap.dot(v.flatten()).reshape((n+1, n+1))
    
    dudt = (U_diffusion + U_reaction).flatten()
    dvdt = (V_diffusion + V_reaction).flatten()
    
    return np.concatenate([dudt, dvdt])

UV0 = np.concatenate([U.flatten(), V.flatten()])

sol = solve_ivp(reaction_diffusion, (tspan[0], tspan[-1]), y0=UV0,
                t_eval=tspan, method='RK45')

A2 = sol.y
print(A2)

[[ 0.70358468  0.27678435 -0.21775865 ... -0.79689015 -0.40972859
   0.07776933]
 [ 0.73241275  0.47188952  0.07344742 ... -0.96577657 -0.78500366
  -0.4261521 ]
 [ 0.81058026  0.37605887 -0.11123233 ... -0.84008598 -0.49565779
  -0.03085913]
 ...
 [ 0.58562756  0.91352592  0.97914313 ... -0.50294695 -0.84298442
  -0.97634716]
 [ 0.6808609   0.87018536  0.97997159 ... -0.16453512 -0.5878894
  -0.88455009]
 [ 0.71061143  0.96093661  0.97601586 ... -0.60413504 -0.91222169
  -0.99697897]]
